# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_enunciados.csv', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Using TensorFlow backend.


Found 15387 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [9]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos2.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 8892 = 57.78904269838175


In [11]:
nok = 0
for word, i in tokenizer.word_index.items():
    if word not in model.wv:
        print(word)
        nok += 1
        if nok > 20: break

666
443
911
711
lo
1952
iec
federativo
887
520
745
sujeitam
527
462
las
abstendo
1960
combatente
pré
732
717


# Treinamento

## Embedding com pesos fixos

In [12]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W0909 12:02:40.340755 139965302404928 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [13]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W0909 12:02:44.557318 139965302404928 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0909 12:02:44.634295 139965302404928 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0909 12:02:44.651153 139965302404928 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0909 12:02:44.651813 139965302404928 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              40962048  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
Total para

In [14]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0909 12:02:55.467239 139965302404928 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 20s 2ms/step - loss: 1.4393 - categorical_accuracy: 0.5979 - val_loss: 1.1403 - val_categorical_accuracy: 0.6132
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.7884 - categorical_accuracy: 0.7756 - val_loss: 1.0179 - val_categorical_accuracy: 0.6981
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.5109 - categorical_accuracy: 0.8583 - val_loss: 1.1325 - val_categorical_accuracy: 0.6958
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.3283 - categorical_accuracy: 0.9077 - val_loss: 1.3637 - val_categorical_accuracy: 0.6902
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.2315 - categorical_accuracy: 0.9373 - val_loss: 1.6459 - val_categorical_accuracy: 0.6793
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1699 - categori

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 18s 2ms/step - loss: 1.1514 - categorical_accuracy: 0.6545 - val_loss: 1.0160 - val_categorical_accuracy: 0.6669
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.3871 - categorical_accuracy: 0.8783 - val_loss: 1.2954 - val_categorical_accuracy: 0.6624
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1302 - categorical_accuracy: 0.9586 - val_loss: 1.7109 - val_categorical_accuracy: 0.6947
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0660 - categorical_accuracy: 0.9805 - val_loss: 2.0672 - val_categorical_accuracy: 0.6763
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0453 - categorical_accuracy: 0.9879 - val_loss: 2.1548 - val_categorical_accuracy: 0.6883
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0295 - categori

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.1658 - categorical_accuracy: 0.6399 - val_loss: 1.0884 - val_categorical_accuracy: 0.6496
Epoch 2/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.5523 - categorical_accuracy: 0.8229 - val_loss: 1.1085 - val_categorical_accuracy: 0.6973
Epoch 3/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.2680 - categorical_accuracy: 0.9176 - val_loss: 1.3142 - val_categorical_accuracy: 0.6932
Epoch 4/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.1532 - categorical_accuracy: 0.9558 - val_loss: 1.5840 - val_categorical_accuracy: 0.6951
Epoch 5/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.0869 - categorical_accuracy: 0.9762 - val_loss: 2.0026 - val_categorical_accuracy: 0.6947
Epoch 6/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.0650 - categori

## Embedding com pesos ajustáveis

In [18]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0909 12:27:48.657403 139965302404928 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0909 12:27:48.685092 139965302404928 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_5 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 2048)              40962048  
_________________________________________________________________
dropout_9 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_10 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                10250     
Total para

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 19s 2ms/step - loss: 1.1451 - categorical_accuracy: 0.6512 - val_loss: 1.2158 - val_categorical_accuracy: 0.6256
Epoch 2/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.3849 - categorical_accuracy: 0.8764 - val_loss: 1.2112 - val_categorical_accuracy: 0.6936
Epoch 3/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.1217 - categorical_accuracy: 0.9649 - val_loss: 1.6386 - val_categorical_accuracy: 0.6962
Epoch 4/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.0609 - categorical_accuracy: 0.9832 - val_loss: 2.0095 - val_categorical_accuracy: 0.6838
Epoch 5/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.0381 - categorical_accuracy: 0.9879 - val_loss: 2.1254 - val_categorical_accuracy: 0.6838
Epoch 6/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.0353 - categori

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 18s 2ms/step - loss: 1.1653 - categorical_accuracy: 0.6352 - val_loss: 0.9994 - val_categorical_accuracy: 0.6891
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.5646 - categorical_accuracy: 0.8193 - val_loss: 1.2584 - val_categorical_accuracy: 0.6583
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.2741 - categorical_accuracy: 0.9126 - val_loss: 1.3575 - val_categorical_accuracy: 0.6925
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1462 - categorical_accuracy: 0.9555 - val_loss: 1.7031 - val_categorical_accuracy: 0.6883
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0909 - categorical_accuracy: 0.9714 - val_loss: 1.8298 - val_categorical_accuracy: 0.7000
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0641 - categori